In [2]:
sys.path.append(r'D:\github\ripple_heterogeneity\python')

import pandas as pd
import numpy as np 
import glob
import matplotlib.pyplot as plt

import sys,os
import seaborn as sns

import functions,loading
import ripple_participation_run

import nelpy as nel  # main nelpy imports

import math
import itertools
from scipy import stats
import multiprocessing
from joblib import Parallel, delayed
from scipy.ndimage import gaussian_filter1d
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
import matplotlib
from matplotlib.ticker import AutoMinorLocator

import importlib

In [ ]:
def session_loop(basepath,save_path):
    save_file = os.path.join(save_path,basepath.replace(os.sep, "_").replace(":", "_")  + '.npy')
    if os.path.exists(save_file):
        return

    cell_metrics,data,ripples,fs_dat = loading.load_basic_data(basepath)

    restrict_idx = ((cell_metrics.putativeCellType == "Pyramidal Cell") &
                        ((cell_metrics.brainRegion=="CA1") |
                        (cell_metrics.brainRegion=="rCA1") |
                        (cell_metrics.brainRegion=="lCA1")) &
                        cell_metrics.bad_unit == False)

    # restrict cell metrics                      
    cell_metrics = cell_metrics[restrict_idx]

    ripple_epochs = nel.EpochArray([np.array([ripples.start,ripples.stop]).T])
    st_unit = nel.SpikeTrainArray(timestamps=np.array(data['spikes'],dtype=object)[restrict_idx], fs=fs_dat)

    unit_mat = get_participation(st_unit[ripple_epochs],ripple_epochs)

    np.save(save_file,unit_mat)


save_path = r'Z:\home\ryanh\projects\ripple_heterogeneity\participation_mats'
parallel = True 
df = pd.read_csv(r'Z:\home\ryanh\projects\ripple_heterogeneity\sessions.csv')

if parallel:
    num_cores = multiprocessing.cpu_count()         
    processed_list = Parallel(n_jobs=num_cores)(delayed(session_loop)(basepath,save_path) for basepath in df.basepath.unique())
else:    
    for basepath in df.basepath.unique():
        print(basepath)
        session_loop(basepath,save_path) 

In [ ]:
def load_in_everything(save_path):
    
    sessions = glob.glob(save_path + os.sep + '*.npy')
    prop_particip = []
    for session in sessions:
        unit_mat = np.load(session)
        prop_particip.append(np.sum(unit_mat,axis=0) / unit_mat.shape[0])

    return prop_particip
    
save_path = r'Z:\home\ryanh\projects\ripple_heterogeneity\participation_mats'
prop_particip = load_in_everything(save_path)   

In [3]:

df = pd.read_csv(r'Z:\home\ryanh\projects\ripple_heterogeneity\sessions.csv')
df = df[df.basepath.str.contains('GrosmarkAD')]
basepath = df.basepath.iloc[0]

cell_metrics,data,ripples,fs_dat = loading.load_basic_data(basepath)

restrict_idx = ((cell_metrics.putativeCellType == "Pyramidal Cell") &
                    ((cell_metrics.brainRegion=="CA1") |
                    (cell_metrics.brainRegion=="rCA1") |
                    (cell_metrics.brainRegion=="lCA1")) &
                    (cell_metrics.bad_unit==False))

# restrict cell metrics                      
cell_metrics = cell_metrics[restrict_idx]

# if cell_metrics.shape[0] == 0:
#     return
    
# get ripple epochs
ripple_epochs = nel.EpochArray([np.array([ripples.start,ripples.stop]).T])
try:
    st = nel.SpikeTrainArray(timestamps=np.array(data['spikes'],dtype=object)[restrict_idx], fs=fs_dat)
except:
    st = nel.SpikeTrainArray(timestamps=np.array(data['spikes'],dtype=object)[restrict_idx][0], fs=fs_dat)


C:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\nelpy\core\_eventarray.py:709: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [5]:
st_rip = st[ripple_epochs]

In [7]:
%time
unit_mat = functions.get_participation(st_rip,ripple_epochs)

Wall time: 0 ns


C:\Users\Cornell\anaconda3\envs\ripple_diversity\lib\site-packages\nelpy\core\_eventarray.py:996: UserWarning: ignoring events outside of eventarray support


In [28]:
importlib.reload(functions)

<module 'functions' from 'D:\\github\\ripple_heterogeneity\\python\\functions.py'>

In [29]:
unit_mat_2 = functions.get_participations2(st_rip.data,
                                        ripple_epochs.starts,
                                        ripple_epochs.stops)


AttributeError: module 'functions' has no attribute 'get_participations'